In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline

import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
# complex valued tensor class
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from models import TorchComplexMLP, ImaginaryDimensionAdder, cplx2tensor, ComplexTorchMLP
from preprocess import *

# Model selection
from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from RegscorePy.bic import bic

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# torch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You're running on", device)

# Doman bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])

N = 15000

DATA_PATH = '../experimental_data/NLS.mat'
data = io.loadmat(DATA_PATH)

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = to_column_vector(Exact_u)
v_star = to_column_vector(Exact_v)

idx = np.random.choice(X_star.shape[0], N, replace=False)

lb = to_tensor(lb, False).to(device)
ub = to_tensor(ub, False).to(device)

X_train = to_tensor(X_star[idx, :], True).to(device)
u_train = to_tensor(u_star[idx, :], False).to(device)
v_train = to_tensor(v_star[idx, :], False).to(device)

feature_names = ['hf', '|hf|', 'h_x', 'h_xx', 'h_xxx']

You're running on cpu


In [3]:
spatial_dim = x.shape[0]
time_dim = t.shape[0]

dt = (t[1]-t[0])[0]
dx = (x[2]-x[1])[0]

fd_h_t = np.zeros((spatial_dim, time_dim), dtype=np.complex64)
fd_h_x = np.zeros((spatial_dim, time_dim), dtype=np.complex64)
fd_h_xx = np.zeros((spatial_dim, time_dim), dtype=np.complex64)
fd_h_xxx = np.zeros((spatial_dim, time_dim), dtype=np.complex64)

for i in range(spatial_dim):
    fd_h_t[i,:] = FiniteDiff(Exact[i,:], dt, 1)
for i in range(time_dim):
    fd_h_x[:,i] = FiniteDiff(Exact[:,i], dx, 1)
    fd_h_xx[:,i] = FiniteDiff(Exact[:,i], dx, 2)
    fd_h_xxx[:,i] = FiniteDiff(Exact[:,i], dx, 3)
    
fd_h_t = np.reshape(fd_h_t, (spatial_dim*time_dim,1), order='F')
fd_h_x = np.reshape(fd_h_x, (spatial_dim*time_dim,1), order='F')
fd_h_xx = np.reshape(fd_h_xx, (spatial_dim*time_dim,1), order='F')
fd_h_xxx = np.reshape(fd_h_xxx, (spatial_dim*time_dim,1), order='F')

In [4]:
inp_dimension = 2
act = CplxToCplx[torch.tanh]
complex_model = CplxSequential(
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 1, bias=True),
                            )

complex_model = torch.nn.Sequential(
                                    torch.nn.Linear(inp_dimension, 200),
                                    RealToCplx(),
                                    complex_model
                                    )

complex_model.load_state_dict(cpu_load("./saved_path_inverse_nls/NLS_cpinn_model.pth"))

/usr/local/lib/python3.9/site-packages/torch/nn/modules/container.py:587: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


<All keys matched successfully>

In [5]:
class ComplexPhysicsInformedNN(nn.Module):
    def __init__(self, model, lb, ub, scale=False):
        super(ComplexPhysicsInformedNN, self).__init__()
        self.model = model
        self.lb = lb
        self.ub = ub
        self.scale = scale
    
    def forward(self, X):
        if self.scale: 
            return self.model(self.neural_net_scale(X))
        return self.model(X)

    def predict(self, X_test):
        return CplxToReal()(self.forward(self.preprocess(*dimension_slicing(X_test))))
    
    def neural_net_scale(self, inp):
        return (2.0*(inp-self.lb)/(self.ub-self.lb))-1.0

    def preprocess(self, spatial, time):
        return cat(spatial, time)
    
    def loss(self, X_f, X0, h0, X_lb, X_ub):
        loss = self.net_f(*dimension_slicing(X_f))
        h0_pred = self.predict(X0); u0 = h0_pred[:, 0:1]; v0 = h0_pred[:, 1:2]
        loss += F.mse_loss(u0, h0[:, 0:1])+F.mse_loss(v0, h0[:, 1:2])
        u_lb, v_lb, u_lb_x, v_lb_x = self.net_h(*dimension_slicing(X_lb))
        u_ub, v_ub, u_ub_x, v_ub_x = self.net_h(*dimension_slicing(X_ub))
        loss += F.mse_loss(u_lb, u_ub)
        loss += F.mse_loss(v_lb, v_ub)
        loss += F.mse_loss(u_lb_x, u_ub_x)
        loss += F.mse_loss(v_lb_x, v_ub_x)
        return loss
    
    def net_h(self, x, t):
        X = cat(x, t)
        h = self.forward(X)
        u = h.real
        v = h.imag
        return u, v, self.diff(u, x), self.diff(v, x)
    
    def net_f(self, x, t):
        u, v, u_x, v_x = self.net_h(x, t)
        u_t, v_t = self.diff(u, t), self.diff(v, t)
        u_xx, v_xx = self.diff(u_x, x), self.diff(v_x, x)
        f_u = u_t + 0.5*v_xx + (u**2 + v**2)*v
        f_v = v_t - 0.5*u_xx - (u**2 + v**2)*u
        return (f_u**2).mean()+(f_v**2).mean()

    def diff(self, func, inp):
        return grad(func, inp, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape, dtype=func.dtype).to(device))[0]

    def complex_mse(self, v1, v2):
        assert v1.shape == v2.shape
        assert v1.shape[1] == 1
        return F.mse_loss(v1.real, v2.real)+F.mse_loss(v2.imag, v2.imag)

    def add_imag_dim(self, v1):
        z = torch.zeros(v1.shape).requires_grad_(False).to(device)
        return torch.complex(v1, z)
    
cpinn = ComplexPhysicsInformedNN(model=complex_model, lb=lb, ub=ub, scale=False).to(device)
cpinn.load_state_dict(cpu_load("./saved_path_inverse_nls/NLS_cpinn.pth"))

<All keys matched successfully>

#### Goals
(1) Re-implement the semisup_model for a complex network.

(2) Implement the self.gradients function.
- complex_model(input) -> diff(u_pred, x) & diff(v_pred, x) -> combine 2 diff terms as 1 complex vector -> compute PDE loss / passing to the selector network

In [6]:
xx, tt = dimension_slicing(to_tensor(X_train, True))
predictions = complex_model(cat(xx, tt))
h = cplx2tensor(predictions)
h_x = complex_diff(predictions, xx)
h_xx = complex_diff(h_x, xx)
h_xxx = complex_diff(h_xx, xx)
h_t = complex_diff(predictions, tt)

/Users/pongpisit/Desktop/Multi-task-Physics-informed-neural-networks/inverse_NLS/../utils.py:112: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(arr).float().requires_grad_(g)


In [7]:
f = 1j*h_t+0.5*h_xx+(h.abs()**2)*h

In [8]:
real_loss = (f.real**2).mean(); imag_loss = (f.imag**2).mean()
avg_loss = (real_loss+imag_loss)*0.5
avg_loss.item()

9.278501238441095e-06

In [9]:
derivatives = to_numpy(cat(h, h.abs()**2, h_x, h_xx, h_xxx))
dictionary = {}
for i in range(len(feature_names)): dictionary[feature_names[i]] = get_feature(derivatives, i)
dictionary

{'hf': array([[ 0.59681845+0.23667818j],
        [-0.44909674+0.17685312j],
        [ 0.879527  +0.68292147j],
        ...,
        [ 0.05834496+0.00590008j],
        [ 0.76030886+0.3223831j ],
        [ 0.29872465+2.2492747j ]], dtype=complex64),
 '|hf|': array([[4.1220880e-01+0.j],
        [2.3296493e-01+0.j],
        [1.2399495e+00+0.j],
        ...,
        [3.4389454e-03+0.j],
        [6.8200046e-01+0.j],
        [5.1484737e+00+0.j]], dtype=complex64),
 'h_x': array([[ 0.27015495-0.00621095j],
        [ 4.0820913 +0.5407842j ],
        [ 0.03203336+1.2565445j ],
        ...,
        [ 0.06036643+0.00272644j],
        [-0.3735373 -0.53195214j],
        [-1.0037583 +2.4175246j ]], dtype=complex64),
 'h_xx': array([[ -0.49388748-0.6165564j ],
        [-13.366825  -1.9379109j ],
        [ -1.3241913 +2.4222994j ],
        ...,
        [  0.08268727+0.00619484j],
        [ -0.47708657+1.0874285j ],
        [  2.104299  -5.629319j  ]], dtype=complex64),
 'h_xxx': array([[-2.0106227e+00-

In [10]:
c_poly = ComplexPolynomialFeatures(feature_names, dictionary)
complex_poly_features = c_poly.fit()
complex_poly_features

Computing hf
Computing |hf|
Computing h_x
Computing h_xx
Computing h_xxx
Computing hf^2
Computing hf |hf|
Computing hf h_x
Computing hf h_xx
Computing hf h_xxx
Computing |hf|^2
Computing |hf| h_x
Computing |hf| h_xx
Computing |hf| h_xxx
Computing h_x^2
Computing h_x h_xx
Computing h_x h_xxx
Computing h_xx^2
Computing h_xx h_xxx
Computing h_xxx^2


array([[ 1.0000000e+00+0.0000000e+00j,  5.9681845e-01+2.3667818e-01j,
         4.1220880e-01+0.0000000e+00j, ...,
        -1.3621695e-01+6.0901898e-01j, -2.0425671e-01+2.1987321e+00j,
         2.7170897e-01+7.8087730e+00j],
       [ 1.0000000e+00+0.0000000e+00j, -4.4909674e-01+1.7685312e-01j,
         2.3296493e-01+0.0000000e+00j, ...,
         1.7491652e+02+5.1807434e+01j, -4.9644681e+02-1.4272940e+02j,
         1.4089141e+03+3.9288962e+02j],
       [ 1.0000000e+00+0.0000000e+00j,  8.7952697e-01+6.8292147e-01j,
         1.2399495e+00+0.0000000e+00j, ...,
        -4.1140513e+00-6.4151759e+00j, -6.2381907e+00-9.9079981e+00j,
        -9.4567375e+00-1.5301050e+01j],
       ...,
       [ 1.0000000e+00+0.0000000e+00j,  5.8344960e-02+5.9000850e-03j,
         3.4389454e-03+0.0000000e+00j, ...,
         6.7988089e-03+1.0244690e-03j,  2.4277112e-03+2.9656675e-04j,
         8.6619466e-04+8.1274120e-05j],
       [ 1.0000000e+00+0.0000000e+00j,  7.6030886e-01+3.2238311e-01j,
         6.8200046e-01

In [11]:
complex_poly_features.shape

(15000, 21)

In [12]:
w = TrainSTRidge(complex_poly_features, to_numpy(h_t), 1e-6, 1000, maxit=100)
print("PDE derived using STRidge")
print_pde(w, c_poly.poly_feature_names)

PDE derived using STRidge
u_t = (-0.000069 +0.499961i)h_xx
    + (-0.000127 +0.999955i)hf |hf|
   


#### Automatic differentiation w/ and w/o Finite difference guidance

In [13]:
class ComplexNetwork(nn.Module):
    def __init__(self, model, index2features=None, scale=False, lb=None, ub=None):
        super(ComplexNetwork, self).__init__()
        # pls init the self.model before
        self.model = model
        # For tracking, the default tup is for the burgers' equation.
        self.index2features = index2features
        print("Considering", self.index2features)
        self.diff_flag = diff_flag(self.index2features)
        self.uf = None
        self.scale = scale
        self.lb, self.ub = lb, ub
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def forward(self, x, t):
        if not self.scale: self.uf = self.model(torch.cat([x, t], dim=1))
        else: self.uf = self.model(self.neural_net_scale(torch.cat([x, t], dim=1)))
        return self.uf
    
    def get_selector_data(self, x, t):
        uf = self.forward(x, t)
        u_t = complex_diff(uf, t)
        
        ### PDE Loss calculation ###
        # Without calling grad
        derivatives = []
        for t in self.diff_flag[0]:
            if t=='hf': 
                derivatives.append(cplx2tensor(uf))
                derivatives.append((uf.real**2+uf.imag**2)+0.0j)
            elif t=='x': derivatives.append(x)
        # With calling grad
        for t in self.diff_flag[1]:
            out = uf
            for c in t:
                if c=='x': out = complex_diff(out, x)
                elif c=='t': out = complex_diff(out, t)
            derivatives.append(out)
        
        return torch.cat(derivatives, dim=-1), u_t
    
    def neural_net_scale(self, inp):
        return 2*(inp-self.lb)/(self.ub-self.lb)-1

In [14]:
complex_network = ComplexNetwork(model=complex_model, index2features=feature_names, scale=True, lb=lb, ub=ub)
X_selector, y_selector = complex_network.get_selector_data(xx, tt)

Considering ['hf', '|hf|', 'h_x', 'h_xx', 'h_xxx']


In [15]:
class ComplexAttentionSelectorNetwork(nn.Module):
    def __init__(self, layers, prob_activation=CplxToCplx[torch.sigmoid], bn=None, reg_intensity=0.1):
        super(ComplexAttentionSelectorNetwork, self).__init__()
        # Nonlinear model, Training with PDE reg.
        assert len(layers) > 1
        self.linear1 = CplxLinear(layers[0], layers[0], bias=True)
        self.prob_activation = prob_activation
        self.nonlinear_model = ComplexTorchMLP(dimensions=layers, bn=bn, dropout_rate=0.1)
        self.latest_weighted_features = None
        self.th = 0.1
        self.reg_intensity = reg_intensity
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        
    def forward(self, inn):
        feature_importances = self.weighted_features(inn)
        inn.real = inn.real*feature_importances.real
        inn.imag = inn.imag*feature_importances.imag
        return self.nonlinear_model(inn)
    
    def weighted_features(self, inn):
        self.latest_weighted_features = self.prob_activation(self.linear1(inn))
        self.latest_weighted_features = self.latest_weighted_features.real.mean(dim=0)+1j*self.latest_weighted_features.imag.mean(dim=0)
        return self.latest_weighted_features
    
    def loss(self, X_input, y_input):
        ut_approx = self.forward(X_input)
        mse_loss = F.mse_loss(ut_approx, y_input, reduction='mean')
        reg_term = F.relu(self.latest_weighted_features-self.th)
        return mse_loss+self.reg_intensity*(torch.norm(reg_term, p=0)+(torch.tensor([1.0, 1.0, 2.0, 3.0, 4.0])*reg_term).sum())

# Only the SemiSupModel has changed to work with the finite difference guidance
class SemiSupModel(nn.Module):
    def __init__(self, network, selector, normalize_derivative_features=False, mini=None, maxi=None):
        super(SemiSupModel, self).__init__()
        self.network = network
        self.selector = selector
        self.normalize_derivative_features = normalize_derivative_features
        self.mini = mini
        self.maxi = maxi
        
    def forward(self, X_u_train, fd_derivatives=None, fd_u_t=None, fd_weight=0.0, include_unsup=True):
        X_selector, y_selector = self.network.get_selector_data(*dimension_slicing(X_u_train))
        
        fd_guidance = 0.0
        if fd_weight>0.0 and fd_derivatives is not None and fd_u_t is not None:
            # Traditional MSE Loss btw uf and u_train + the fd_guidance loss
            row, col = fd_derivatives.shape
            fd_guidance += F.mse_loss(X_selector[:row, 0:1], fd_derivatives[:, 0:1])
            fd_guidance += fd_weight*(col-1)*F.mse_loss(X_selector[:row, 1:], fd_derivatives[:, 1:])
            fd_guidance += fd_weight*F.mse_loss(y_selector[:row, :], fd_u_t)
            
        else: fd_guidance = self.network.uf
            
        if self.normalize_derivative_features:
            X_selector = (X_selector-self.mini)/(self.maxi-self.mini)
        
        if include_unsup: unsup_loss = self.selector.loss(X_selector, y_selector)
        else: unsup_loss = None
        
        return fd_guidance, unsup_loss

In [16]:
selector = ComplexAttentionSelectorNetwork([len(feature_names), 50, 50, 1], prob_activation=CplxToCplx[F.softmax](), bn=None)
selector(X_selector)

/usr/local/lib/python3.9/site-packages/torch/nn/modules/container.py:587: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")
/usr/local/lib/python3.9/site-packages/cplxmodule/cplx.py:197: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return type(self)(f(self.__real, *a, **k), f(self.__imag, *a, **k))


Cplx(
  real=tensor([[0.1009],
        [0.0776],
        [0.0835],
        ...,
        [0.0387],
        [0.0910],
        [0.0864]], grad_fn=<AddBackward0>),
  imag=tensor([[-0.1251],
        [-0.1170],
        [-0.1380],
        ...,
        [-0.0967],
        [-0.1335],
        [-0.1413]], grad_fn=<AddBackward0>)
)

In [18]:
# semisup_model = SemiSupModel(network=Network(
#                                     model=TorchMLP(dimensions=[2, 50, 50, 50 ,50, 50, 1],
#                                                    activation_function=nn.Tanh,
#                                                    bn=nn.LayerNorm, dropout=None),
#                                     index2features=feature_names, scale=True, lb=lb, ub=ub),
#                             selector=AttentionSelectorNetwork([len(feature_names), 50, 50, 1], prob_activation=nn.Softmax(dim=1), bn=nn.LayerNorm),
#                             normalize_derivative_features=True,
#                             mini=None,
#                             maxi=None)